# Installing reqired libs


In [ ]:
!unzip results.zip

Archive:  results.zip
replace __huggingface_repos__.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
  inflating: flan-t5-lora-wikisimplification-stage1-small/checkpoint-2326/README.md  
  inflating: flan-t5-lora-wikisimplification-stage1-small/checkpoint-2326/adapter_config.json  
  inflating: flan-t5-lora-wikisimplification-stage1-small/checkpoint-2326/adapter_model.safetensors  
  inflating: flan-t5-lora-wikisimplification-stage1-small/checkpoint-2326/optimizer.pt  
  inflating: flan-t5-lora-wikisimplification-stage1-small/checkpoint-2326/rng_state.pth  
  inflating: flan-t5-lora-wikisimplification-stage1-small/checkpoint-2326/scheduler.pt  
  inflating: flan-t5-lora-wikisimplification-stage1-small/checkpoint-2326/special_tokens_map.json  
  inflating: flan-t5-lora-wikisimplification-stage1-small/checkpoint-2326/spiece.model  
  inflating: flan-t5-lora-wikisimplification-stage1-small/checkpoint-2326/tokenizer.json  
  inflating: flan-t5-lora-wikisimplification-stage1-small/checkpoint-2

In [ ]:
!pip install -U transformers datasets peft accelerate


In [ ]:
import pandas as pd
df = pd.read_csv("training_data_7213.csv")
df.head()

,medical,layman
0,The 1-year mortality rate in this cohort was 1...,About 18% of people in this group died within ...
1,"The 1/Pcr value, calculated as the reciprocal ...","The 1/Pcr number, which is the inverse of the ..."
2,Serum 3‑beta‑hydroxybutyrate levels rise durin...,"During a long fast, the liver produces more ke..."
3,A bulge in the fovea is a focal elevation of t...,A bulge in the fovea is a small raised spot in...
4,The patient exhibits a progressive disease cha...,The illness keeps getting worse over time.


In [ ]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
)
from peft import PeftModel

BASE_MODEL = "google/flan-t5-base"
ADAPTER_PATH = "/content/flan-t5-lora-wikisimplification-stage1/final_lora_adapters"
CSV_PATH = "/content/training_data_7213.csv"   # <-- upload your CSV here
OUTPUT_DIR = "/content/stage2_medical_lora"

PROMPT = "Simplify:\n"

MAX_SOURCE_LEN = 512
MAX_TARGET_LEN = 128


In [ ]:
df = pd.read_csv(CSV_PATH)

# build supervised pairs
inputs = [PROMPT + x for x in df["medical"].astype(str)]
outputs = df["layman"].astype(str).tolist()

dataset = Dataset.from_dict({
    "input": inputs,
    "output": outputs,
})

dataset


Dataset({
    features: ['input', 'output'],
    num_rows: 7213
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

model = AutoModelForSeq2SeqLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


In [ ]:
model = PeftModel.from_pretrained(
    model,
    ADAPTER_PATH,
    is_trainable=True,
)

model.print_trainable_parameters()


trainable params: 2,310,144 || all params: 249,888,000 || trainable%: 0.9245


In [ ]:
def preprocess(batch):
    model_inputs = tokenizer(
        batch["input"],
        max_length=MAX_SOURCE_LEN,
        truncation=True,
        padding="max_length",
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["output"],
            max_length=MAX_TARGET_LEN,
            truncation=True,
            padding="max_length",
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
dataset = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset.column_names,
)


Map:   0%|          | 0/7213 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=1e-4,
    num_train_epochs=5,
    warmup_steps=100,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    bf16=torch.cuda.is_available(),
    fp16=not torch.cuda.is_available(),
    remove_unused_columns=False,
    report_to="none",
)


In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)


The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
trainer.train()
trainer.save_model(OUTPUT_DIR)


Step,Training Loss
50,19.610700
100,12.361500
150,4.100500
200,3.266400
250,2.574700
300,2.034400
350,1.701300
400,1.560700
450,1.554000
500,1.483800


Step,Training Loss
50,19.610700
100,12.361500
150,4.100500
200,3.266400
250,2.574700
300,2.034400
350,1.701300
400,1.560700
450,1.554000
500,1.483800


In [ ]:
!zip -r stage2_medical_lora.zip /content/stage2_medical_lora


  adding: content/stage2_medical_lora/ (stored 0%)
  adding: content/stage2_medical_lora/training_args.bin (deflated 54%)
  adding: content/stage2_medical_lora/README.md (deflated 66%)
  adding: content/stage2_medical_lora/tokenizer.json (deflated 74%)
  adding: content/stage2_medical_lora/adapter_model.safetensors (deflated 7%)
  adding: content/stage2_medical_lora/special_tokens_map.json (deflated 85%)
  adding: content/stage2_medical_lora/checkpoint-2255/ (stored 0%)
  adding: content/stage2_medical_lora/checkpoint-2255/training_args.bin (deflated 54%)
  adding: content/stage2_medical_lora/checkpoint-2255/trainer_state.json (deflated 75%)
  adding: content/stage2_medical_lora/checkpoint-2255/README.md (deflated 66%)
  adding: content/stage2_medical_lora/checkpoint-2255/tokenizer.json (deflated 74%)
  adding: content/stage2_medical_lora/checkpoint-2255/adapter_model.safetensors (deflated 7%)
  adding: content/stage2_medical_lora/checkpoint-2255/special_tokens_map.json (deflated 85%)
